In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score, mean_absolute_error, max_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor


In [ ]:
mydata = pd.read_csv('/content/SolarPrediction.csv')

In [ ]:
mydata.head()

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 0:00,23:55:26,1.21,48,30.46,59,177.39,5.62,6:13:00,18:13:00
1,1475229023,9/29/2016 0:00,23:50:23,1.21,48,30.46,58,176.78,3.37,6:13:00,18:13:00
2,1475228726,9/29/2016 0:00,23:45:26,1.23,48,30.46,57,158.75,3.37,6:13:00,18:13:00
3,1475228421,9/29/2016 0:00,23:40:21,1.21,48,30.46,60,137.71,3.37,6:13:00,18:13:00
4,1475228124,9/29/2016 0:00,23:35:24,1.17,48,30.46,62,104.95,5.62,6:13:00,18:13:00


In [ ]:
mydata['Year'] = pd.DatetimeIndex(mydata['Data']).year
mydata['Month'] = pd.DatetimeIndex(mydata['Data']).month
mydata['Day'] = pd.DatetimeIndex(mydata['Data']).day


In [ ]:
mydata['Hour'] = pd.DatetimeIndex(mydata['Time']).hour
mydata['Minute'] = pd.DatetimeIndex(mydata['Time']).minute
mydata['Second'] = pd.DatetimeIndex(mydata['Time']).second


In [ ]:
mydata['SunPerDay'] = pd.DatetimeIndex(mydata['TimeSunSet']) - pd.DatetimeIndex(mydata['TimeSunRise'])
mydata['SunPerDayHours'] = pd.DatetimeIndex(mydata['TimeSunSet']).hour - pd.DatetimeIndex(mydata['TimeSunRise']).hour


In [ ]:
mydata.drop(['Time', 'Data', 'TimeSunRise', 'TimeSunSet', 'SunPerDay'], axis=1, inplace=True)


In [ ]:
mydata.isnull().sum()


,0
UNIXTime,0
Radiation,0
Temperature,0
Pressure,0
Humidity,0
WindDirection(Degrees),0
Speed,0
Year,0
Month,0
Day,0


In [ ]:
mydata.info()
mydata.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Radiation               32686 non-null  float64
 2   Temperature             32686 non-null  int64  
 3   Pressure                32686 non-null  float64
 4   Humidity                32686 non-null  int64  
 5   WindDirection(Degrees)  32686 non-null  float64
 6   Speed                   32686 non-null  float64
 7   Year                    32686 non-null  int32  
 8   Month                   32686 non-null  int32  
 9   Day                     32686 non-null  int32  
 10  Hour                    32686 non-null  int32  
 11  Minute                  32686 non-null  int32  
 12  Second                  32686 non-null  int32  
 13  SunPerDayHours          32686 non-null  int32  
dtypes: float64(4), int32(7), int64(3)
memo

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,Year,Month,Day,Hour,Minute,Second,SunPerDayHours
count,3.268600e+04,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000,32686.0,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000,32686.000000
mean,1.478047e+09,207.124697,51.103255,30.422879,75.016307,143.489821,6.243869,2016.0,10.526066,15.825766,11.557425,27.556691,17.441902,11.347886
std,3.005037e+06,315.916387,6.201157,0.054673,25.990219,83.167500,3.490474,0.0,1.096691,8.711824,6.912034,17.268507,12.951109,0.476307
min,1.472724e+09,1.110000,34.000000,30.190000,8.000000,0.090000,0.000000,2016.0,9.000000,1.000000,0.000000,0.000000,0.000000,11.000000
25%,1.475546e+09,1.230000,46.000000,30.400000,56.000000,82.227500,3.370000,2016.0,10.000000,9.000000,6.000000,15.000000,5.000000,11.000000
50%,1.478026e+09,2.660000,50.000000,30.430000,85.000000,147.700000,5.620000,2016.0,11.000000,16.000000,12.000000,30.000000,18.000000,11.000000
75%,1.480480e+09,354.235000,55.000000,30.460000,97.000000,179.310000,7.870000,2016.0,11.000000,23.000000,18.000000,45.000000,22.000000,12.000000
max,1.483265e+09,1601.260000,71.000000,30.560000,103.000000,359.950000,40.500000,2016.0,12.000000,31.000000,23.000000,57.000000,59.000000,12.000000


In [ ]:
X = mydata.drop(columns=['Radiation'])
y = mydata['Radiation']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)


dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train_scaled, y_train)


xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(X_train_scaled, y_train)




XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
lr_preds = lr_model.predict(X_test_scaled)
rf_preds = rf_model.predict(X_test_scaled)
dt_preds = dt_model.predict(X_test_scaled)
xgb_preds = xgb_model.predict(X_test_scaled)


In [ ]:
print("Linear Regression R2:", r2_score(y_test, lr_preds))
print("Random Forest R2:", r2_score(y_test, rf_preds))
print("Decision Tree R2:", r2_score(y_test, dt_preds))
print("XGBoost R2:", r2_score(y_test, xgb_preds))


Linear Regression R2: 0.6239417259189338
Random Forest R2: 0.935821204522368
Decision Tree R2: 0.8821631791579465
XGBoost R2: 0.9285813931762983


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score, max_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Sample data (replace this with your actual dataset)
X, y = mydata.drop(columns=['Radiation']), mydata['Radiation']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train old model (e.g., Linear Regression)
old_model = LinearRegression()
old_model.fit(X_train, y_train)
y_pred_old = old_model.predict(X_test)

# Train new model (e.g., Random Forest)
new_model = RandomForestRegressor()
new_model.fit(X_train, y_train)
y_pred_new = new_model.predict(X_test)

# Calculate metrics for the old model
mae_old = mean_absolute_error(y_test, y_pred_old)
mse_old = mean_squared_error(y_test, y_pred_old)
r2_old = r2_score(y_test, y_pred_old)
variance_old = explained_variance_score(y_test, y_pred_old)
max_err_old = max_error(y_test, y_pred_old)

# Calculate metrics for the new model
mae_new = mean_absolute_error(y_test, y_pred_new)
mse_new = mean_squared_error(y_test, y_pred_new)
r2_new = r2_score(y_test, y_pred_new)
variance_new = explained_variance_score(y_test, y_pred_new)
max_err_new = max_error(y_test, y_pred_new)

# Display the results
print("Old Parameters ------- New Parameters")
print(f"MAE : {mae_old} ------- {mae_new}")
print(f"MSE : {mse_old} ------- {mse_new}")
print(f"R^2 : {r2_old} ------- {r2_new}")
print(f"Var : {variance_old} ------- {variance_new}")
print(f"Max : {max_err_old} ------- {max_err_new}")


Old Parameters ------- New Parameters
MAE : 145.93109764307636 ------- 29.732634551850715
MSE : 37360.18564879768 ------- 6314.876231336868
R^2 : 0.6239417259192305 ------- 0.9364360370445151
Var : 0.6240038428068435 ------- 0.9364475974418769
Max : 919.828366060648 ------- 870.1324000000004
